In [63]:
import pandas as pd

In [64]:
def load_data(file_name):
    return pd.read_excel(file_name,engine="openpyxl",header = None)

In [65]:
def pre_cleaning(df):
    df.columns = ["job_name","link","3","4","5","6"]
    df = df.fillna(value="Private Advertiser").rename(columns = {"3":"Company"})
    df['ad_type'] = df['link'].str.extract('\=(\w+)\#')
    return df

In [66]:
def listing_time_cleaning(df):
    df['listing_time_indays'] = df['5'].str.extract('(\d+[a-z])')
    df['listing_time_indays'] = df['listing_time_indays'].str.extract('(\w+)d')
    df['listing_time_indays'].fillna('0', inplace = True)
    df['listing_time_indays'] #notice that if the ad_type is promoted, the listing time is recorded as "Featured", which means the listing times of those featured advs may be longer.
    return df

In [67]:
#useful function
def deduplication(x):
    mid = len(x)//2
    if x != 'unknown':
        return x[:mid+1]
    else:
        return 'unknown'
def smart_deduplication(x):
    trim = x.strip()
    index = (trim+trim).find(trim,1)
    if index != -1:
        return trim[:index]
    else:
        return 'unknown'

In [68]:
def location_cleaning(df):
    df[['location','area']] = df['4'].str.split("area:",expand= True)
    df['location'] = df['location'].apply(lambda x:x[9:])
    df['location'] = df['location'].apply(smart_deduplication)
    df['area'].fillna('unknown', inplace = True)
    df['area'] = df['area'].apply(smart_deduplication)
    return df

In [69]:
def classification_cleaning(df):
    df[['subclassification','Classification']] = df['6'].str.split('subClassification:',expand=True)
    df['subclassification'] = df['subclassification'].apply(lambda x:x[15:])
    df['Classification'].fillna('unknown', inplace = True)
    df['Classification'] = df['Classification'].apply(smart_deduplication)
    df[['mainClassification','7','subclassification']] = df['subclassification'].str.split('&',expand=True)
    df['subclassification'].fillna('unknown', inplace = True)
    return df

In [70]:
def drop_columns(df):
    df.drop('4', inplace=True, axis=1)
    df.drop('5', inplace=True, axis=1)
    df.drop('6', inplace=True, axis=1)
    df.drop('7', inplace=True, axis=1)
    return df

In [71]:
Admin_cleaning = (df.pipe(pre_cleaning).
    pipe(listing_time_cleaning).
    pipe(location_cleaning).
    pipe(classification_cleaning).
    pipe(drop_columns)
)


In [72]:
df = load_data("NZ_Admin_JOBS.xlsx") #input the file name
Admin_cleaning

,job_name,link,Company,ad_type,listing_time_indays,location,area,subclassification,Classification,mainClassification
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Private Advertiser,promoted,0,Bay of Plenty,Tauranga,Office Support,Office Management,Administration
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,promoted,0,Bay of Plenty,Tauranga,Office Support,Receptionists,Administration
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,standard,4,Auckland,unknown,Office Support,Other,Administration
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,standard,0,Southland,Invercargill,Office Support,Administrative Assistants,Administration
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Private Advertiser,standout,4,Canterbury,Christchurch,Office Support,Client & Sales Administration,Administration
...,...,...,...,...,...,...,...,...,...,...
2703,Key Account Manager,https://www.seek.co.nz/job/50490062?type=stand...,Hays Talent Solutions,standout,27,Auckland,Auckland Central,Office Support,Client & Sales Administration,Administration
2704,Executive Assistant,https://www.seek.co.nz/job/50488000?type=stand...,one eighty recruitment,standout,27,Wellington,"Wellington CentralWellington Central,Competiti...",unknown,unknown,rly rate
2705,Temporary Office Roles,https://www.seek.co.nz/job/50524865?type=stand...,Asset Recruitment Ltd,standout,20,Waikato,"HamiltonHamilton,Competitive hourly rates $$",unknown,unknown,rly rates $$
2706,Temporary Office Roles,https://www.seek.co.nz/job/50477118?type=stand...,Asset Recruitment Ltd,standout,28,Waikato,"HamiltonHamilton,Competitive hourly rates $$",unknown,unknown,rly rates $$
